# How to Use:
This Notebook should ease the use of the API for "cleanup" tasks. It includes some sample for specific use cases that can be adjusted.
1. run first block to prepare excecutions by creating api connection
2. run any following block to execute a specific function

In [ ]:
import logging
from churchtools_api.churchtools_api import ChurchToolsApi

logging.getLogger().setLevel(logging.INFO)
logging.info("Started main")

# Create Session
from secure.config import ct_domain
from secure.config import ct_token
api = ChurchToolsApi(ct_domain)
api.login_ct_rest_api(ct_token=ct_token)

## Song Tag - mass change
Helper to append a tag to all songs - e.g. 51 is tag:"in ChurchTools vor Skript Import"

In [ ]:
songs = api.get_songs()
all_song_ids = [value['id'] for value in songs]
for id in all_song_ids:
    api.add_song_tag(id, 51)